# Defect Classifications of AOI

In [60]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
import os
from torchvision import transforms

## 1. Data preprocess

In [26]:
test_df = pd.read_csv('aoi/test.csv')
test_df.loc[0, "Label"]

nan

In [92]:
class CustomDataset(Dataset):
    def __init__(self, csv_path, images_folder, transform = False):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        filename = self.df.loc[index, "ID"]
        label = self.df.loc[index, "Label"].item()
        image = Image.open(os.path.join(self.images_folder, filename))
        if self.transform:
            image = self.transform(image)
        return image, label

In [93]:
train_transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])

In [94]:
train_dataset = CustomDataset('aoi/train.csv','aoi/train_images', transform=train_transform)
test_dataset = CustomDataset('aoi/test.csv','aoi/test_images', transform=test_transform)
train_dataloader = DataLoader(train_dataset, batch_size=18, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=18, shuffle=False)   

In [96]:
for images, labels in train_dataloader:
    print(images.shape)
    print(labels.shape)
    break

torch.Size([18, 1, 512, 512])
torch.Size([18])


## 2. Model architecture

## 3. Training

## 4. Validation